In [3]:
from mango.training.trackers import NeptuneTracker

In [4]:
import os
tracker = NeptuneTracker('mango/mango-noise',os.getenv('NEPTUNE_API_TOKEN'), 'placeholder-clf')

[neptune] [warning] NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/


[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/mango/mango-noise/e/MNGNS-7


In [1]:
from mango.utils.multilabel import render_confusion_matrix
import numpy as np

imgs = render_confusion_matrix(np.random.randint(0, 2, (16, 10)), np.random.randint(0, 2, (16, 10)))

In [2]:
tracker.init_run('temp2', {'base_lr': 11, 'num_layers': 10})
tracker.log_epoch({'loss/eval': 11, 'acc/eval': 0.3, 'imgs': imgs}, 0)
tracker.log_epoch({'loss/eval': 10, 'acc/eval': 0.4, 'imgs': imgs}, 1)
tracker.log_epoch({'loss/eval': 10, 'acc/eval': 0.45, 'imgs': imgs}, 2)
tracker.log_epoch({'loss/eval': 7, 'acc/eval': 0.52, 'imgs': imgs}, 3)
tracker.log_epoch({'loss/eval': 6.1, 'acc/eval': 0.6, 'imgs': imgs}, 4)

NameError: name 'tracker' is not defined

In [3]:
import torch
model = torch.nn.Sequential(torch.nn.Linear(100, 200), torch.nn.ReLU(), torch.nn.Linear(200, 64))

In [ ]:
tracker.save_model(model, 0)
tracker.save_model(model, 1)

In [4]:
from datasets import load_dataset
from transformers import WhisperProcessor
from mango.utils.tokenization import retain_cyrillic
from mango.training.SpeakerAttributedMixer import SpeakerAttributedMixer, DatasetMixerConfig
from mango.training.DatasetMixer import DatasetMixerWrapper
from mango.training.collators import WhisperUrbanCollator
from mango.models.noiseclas import WhisperTinyEmbedder, NoiseClassifier, ClassificationMulticlassAccuracy
from mango.training.MangoTrainer import MangoTrainer, TrainerConfig
from torch.utils.data import DataLoader

In [5]:
cv13 = load_dataset('mozilla-foundation/common_voice_13_0', 'uk', trust_remote_code=True)
#urban = load_dataset('danavery/urbansound8K')
urban = load_dataset('anakib1/audioset', split='train')

Generating train split:   0%|          | 0/4029 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1107 [00:00<?, ? examples/s]

In [6]:
cv13 = cv13['train'].rename_columns({'sentence': 'transcription', 'client_id': 'speaker_id'})
cv13 = retain_cyrillic(cv13, 'transcription')

In [7]:
cv13.set_format('pt')
urban.set_format('pt')

In [8]:
train_mixer = SpeakerAttributedMixer(
    DatasetMixerConfig(max_speakers=3, utterances_count=150, beta=5, min_repetitions=3, max_repetitions=6, noise_sample_type='random'), cv13,
    urban)
test_mixer = SpeakerAttributedMixer(
    DatasetMixerConfig(max_speakers=3, utterances_count=50, beta=5, min_repetitions=3, max_repetitions=6, noise_sample_type='random'), cv13,
    urban)

train_data = DatasetMixerWrapper(train_mixer)
test_data = DatasetMixerWrapper(test_mixer)